In [ ]:
import pandas as pd # pandas will be referred to as pd below
import numpy
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')

Load files: Data comes from two different csv files: one for GDP data and one for Mortality Rate.

GDP Data...

In [ ]:
data_GDP= pd.read_csv('GDP.csv')
data_GDP.columns

In [ ]:
#Selecting columns:
data_GDP['Area'].head(3)

In [ ]:
#Selecting rows:
data_GDP[:5]

In [ ]:
#Selecting columns & rows:
data_GDP['Area'][:5]

In [ ]:
#Selecting multiple columns:
data_GDP[['Area','GDP']][:5]

In [ ]:
data_GDP.loc[0,"GDP"]

In [ ]:
type(data_GDP.loc[0,"GDP"])

In [ ]:
# Remove missing values: 
# dropna():dropped DataFrame
print('GDP original length: '+ str(len(data_GDP)))
data_GDP_NoMissing= data_GDP.dropna() 

# default drop method: if all values are NA, drop that label
print('GDP new length: '+ str(len(data_GDP_NoMissing)))

# OR, You can replace NA with a scalar value:
# data_GDP= data_GDP.fillna(0)

# You can use pd.isnull and pd.notnull to test for missing data (NaN).

# be careful about the indexes:
data_GDP_NoMissing.index # note that the index does not start from zero!





In [ ]:
data_GDP_NoMissing.iloc[0] # by index position
# data_GDP_NoMissing.loc[0] # by index label (error if such a label does not exist)
# you can set the index as you like:

In [ ]:
data_GDP_NoMissing.index= range(len(data_GDP_NoMissing))
data_GDP_NoMissing.index

In [ ]:
# A note on numpy.nan: recode missing values to python missing (NaN)
# sub2['S3AQ3B1']=sub2['S3AQ3B1'].replace(9, numpy.nan)

MortalityRate Data...

In [ ]:
data_MortalityRate= pd.read_csv('MortalityRate.csv')
print(data_MortalityRate.columns)

In [ ]:
print('Mortality rate original length: '+ str(len(data_MortalityRate)))

data_MortalityRate_NoMissing= data_MortalityRate.dropna()
print('Mortality rate new length: '+ str(len(data_MortalityRate_NoMissing)))

Merge two data frames:
Inner joins yield a DataFrame that contains only rows where the value being joined (acc to column names) exists in BOTH tables. 
Now, data contains no empty rows

In [ ]:
data = pd.merge(left= data_GDP_NoMissing, right= data_MortalityRate_NoMissing)
print (data.head(5))

In [ ]:
print('Data length: '+ str(len(data)))

In [ ]:
data.index

In [ ]:
# Alternatively, you could do dropna like below:
data = pd.merge(left= data_GDP, right= data_MortalityRate)
print (data.head(5))
print('Lenghth of the original dataframe: '+str(len(data)))
data= data.dropna(subset =['GDP','Under_five_mortality'], how='any')

data= data.reset_index(drop=True) # not to save old index as a column
print('Lenghth of the new dataframe: '+str(len(data)))
print(data.head(3))
data.index

In [ ]:
type(data.loc[0,"GDP"])

In [ ]:
type(data.loc[0,"Year"])

In [ ]:
data.columns

Converts to numeric :lets python recognize these columns as numeric so that some numeric operations can be done

In [ ]:
data['Year']= pd.to_numeric(data['Year'])
data['GDP']= pd.to_numeric(data['GDP'])
data['Under_five_mortality']= pd.to_numeric(data['Under_five_mortality'])

Frequency distributions...

For column: Area, Counts and Percentages

In [ ]:
# Which area has the highest number of records?
countArea= data['Area'].value_counts(sort= True, ascending=False, dropna=False) # sort= True
print(countArea.head(5))
print()
perArea= data['Area'].value_counts(sort= True, ascending=False, dropna=False, normalize=True)
print(perArea.head(5))
print()
print ('Number of Different Areas: '+ str(len(countArea)))
print ('Number of observations for the Area column:'+ str(len(data['Area'])))


For column: Year, Counts and Percentages

In [ ]:
print ('Year ranges from '+ str(data['Year'].min())+' and to '+ str(data['Year'].max()) )
countYear= data['Year'].value_counts(sort= False, dropna=False)
print(countYear.head(5))
perYear= data['Year'].value_counts(sort= False, dropna=False, normalize=True)
print(perYear.head(5))
print ('Number of Different Years: '+ str(len(countYear)))

For column: GDP, Counts and Percentages

In [ ]:
countGDP= data['GDP'].value_counts(sort= False, dropna=False) 
# dropna=False  required, otherwise python does not show mising values

print(countGDP.head(5))
perGDP= data['GDP'].value_counts(sort= False, dropna=False, normalize=True)
print(perGDP.head(5))
print ('Number of Different GDP values: '+ str(len(countGDP)))

For column: Under_five_mortality, Counts and Percentages

In [ ]:
countMortality= data['Under_five_mortality'].value_counts(sort= False, dropna=False)
print(countMortality.head(5))
perMortality= data['Under_five_mortality'].value_counts(sort= False, dropna=False, normalize=True)
print(perMortality.head(5))
print ('Number of Different mortality rates: '+ str(len(countMortality)))

Manipulate data:
Remove rows where either GDP or Under_five_mortality column is NaN: Should return the same dataframe, because above merge operation has already done this!

In [ ]:
data= data.dropna(subset =['GDP','Under_five_mortality'], how='any')

data= data.reset_index(drop=True) # not to save old index as a column
print('Lenghth of the new dataframe: '+str(len(data)))

# Now, Create GDP groups, describe and visualize data:

In [ ]:
print ('GDP ranges from '+ str(data['GDP'].min())+' and to '+ str(data['GDP'].max()) )
data['GDP'].describe()

In [ ]:
# GDP column keeps continuous quantitative variables.
# In order to create GDP groups, we need to generate partitions. 
# Below, we call the "cut " method. Note that the first cut is (280, 1000]
data['GDPGroup']= pd.cut(data.GDP, [280,1000,2000,5000,10000,50000,120000]) 
# no need to sort the data frame.

print(len(data))
print(len(data['GDPGroup']))


In [ ]:
data['GDPGroup'].head(10)

Frequency distributions for the group...

In [ ]:
data['GDPGroup']= data['GDPGroup'].astype('category')
data['GDPGroup'].describe()
# "GDPGroup" column keeps a categorial variable (not quantitative):
# astype() method: Cast object to input numpy.dtype

In [ ]:
data['GDPGroup'].head(10)

In [ ]:
GDPGroupP= data['GDPGroup'].value_counts(sort= False, normalize= True) 
# value_counts() should make more sense now
print(GDPGroupP)

In [ ]:
print(pd.crosstab(data['GDPGroup'], data['GDP'])) 
# crosstab() method: shows which GDP valus were put into which GDP group

## Univariate (related to only 1 variable) bar graph for categorical GDPGroup varaible:

In [ ]:
seaborn.countplot(x= 'GDPGroup', data= data)
plt.xlabel('GDP (per capita) categories)')
plt.title('Counts of rows for each GDP')
plt.show()

Univariate histogram for quantitative GDP varaible:

In [ ]:
seaborn.distplot(data['GDP'].dropna(), kde= False)
plt.xlabel('GDP (per capita) quantitative values)')
plt.title('Counts of rows for each GDP')

In [ ]:
plt.hist(data.GDP,bins=5)

# Create Under_five_mortality groups, describe and visualize:

In [ ]:
print ('Under_five_mortality ranges from '+ str(data['Under_five_mortality'].min())+' and to '+ 
       str(data['Under_five_mortality'].max()) )
data['Under_five_mortality'].describe()

In [ ]:
data['MortalityGroup']= pd.cut(data.Under_five_mortality, [2,50,100,200,500,800]) 

In [ ]:
data['MortalityGroup']= data['MortalityGroup'].astype('category')
data['MortalityGroup'].describe()

In [ ]:
MG= data['MortalityGroup'].value_counts(sort= False, normalize= True)
print(MG)

In [ ]:
seaborn.countplot(x= 'MortalityGroup', data= data)
plt.xlabel('Mortality Rate group')
plt.title('Counts of rows for each Rate')
plt.show()

In [ ]:
seaborn.regplot(x= "GDP", y= "Under_five_mortality", fit_reg= False, data= data)

Use data for only years after 2000

In [ ]:
data2= data[data['Year']>=2000]

In [ ]:
seaborn.regplot(x= "GDP", y= "Under_five_mortality", fit_reg= False, data= data2)

Create a factor plot, but first convert the "Under_five_mortality" variable to a two-level categorical variable:

If the Under_five_mortality is above 50 (per 1000) then I will accept it as 1:

In [ ]:
MG= data['MortalityGroup'].value_counts(sort= False, normalize= True)
print(MG)

In [ ]:
def MORTAL (row):
    if row['Under_five_mortality'] < 50 :
        return 0
    else:
        return 1

In [ ]:
data2['Mortal']= data2.apply(lambda row : MORTAL(row), axis= 1)
# axis=1, tells python to apply this function to each row 
# Arbitrary functions can be applied along the axes of a DataFrame using the apply() method
data2['Mortal'].value_counts()

In [ ]:
seaborn.factorplot(x= 'GDPGroup', y= 'Mortal', data= data2, kind= "bar", ci=None)
plt.xlabel('GDP per capita group')
plt.ylabel('proportion mortality rate')
plt.show()
# A categorical to categorical bar chart
# note that a bivariate graph displays a mean on the y-axis, 
# so categorical response variables should not have other than two levels, they should be coded as 1 and 0.
# Important: Below, the response variable gives us the proportion of the positive observations.

Rename the groups for better readability: 

In [ ]:
def GDPRangeGroup (row):
   if row['GDP'] > 280.0 and row['GDP']<= 1000.0 :
      return 1
   if row['GDP'] > 1000.0 and row['GDP']<= 2000.0 :
      return 2
   if row['GDP'] > 2000.0 and row['GDP']<= 5000.0 :
      return 3
   if row['GDP'] > 5000.0 and row['GDP']<= 10000.0 :
      return 4
   if row['GDP'] > 10000.0 and row['GDP']<= 50000.0 :
      return 5
   if row['GDP'] > 50000.0 and row['GDP']<= 120000.0 :
      return 6

In [ ]:
data2['GDPCat'] = data2.apply (lambda row: GDPRangeGroup (row),axis=1)

In [ ]:
seaborn.factorplot(x= 'GDPCat', y= 'Mortal', data= data2, kind= "bar", ci=None)
plt.xlabel('GDP per capita category: 1 the most poor to 6 the richest')
plt.ylabel('Proportion mortality')
plt.show()

# PART II of the course

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 

In [ ]:
data3= data2[['GDPCat', 'Under_five_mortality']]
print(data3.head(15))

## A) ANOVA 
### Please note that "GDPCat" is categorical, and  "Under_five_mortality" is numeric

In [ ]:
model1= smf.ols(formula='Under_five_mortality ~ C(GDPCat)', data=data3)
# statsmodels.formula.api as smf
# ols: ordinary least squares regression
# first response var, and then explanatory variable. Note that C stands for the Categorical variable.

In [ ]:
results= model1.fit()

In [ ]:
print(results.summary())

### POST-HOC TESTS ARE NEEDED: Which categories are different than others?

In [ ]:
mc1 = multi.MultiComparison(data3['Under_five_mortality'], data3['GDPCat'])
res1 = mc1.tukeyhsd()
print(res1.summary())

### B) CHI SQUARE TEST OF INDEPENDENCE for CATEGORICAL TO CATEGORICAL VARIABLES

In [ ]:
data4= data2[['Mortal', 'GDPCat']]
print(data4[data4.Mortal==0].head())

In [ ]:
# contingency table of observed counts
ct1=pd.crosstab(data4['Mortal'], data4['GDPCat'])
print (ct1) # ct1 is a two-dimentional array

In [ ]:
# column percentages
colsum=ct1.sum(axis=0) # axis=0 to sum all columns
colpct=ct1/colsum
print(colpct)


#### We are trying to find out if the mortality rate among different GDP categories is the same or not. Therefore column percentages make sense. (Explanatory variables on each column)

In [ ]:
import scipy.stats

In [ ]:
# chi-square
print ('chi-square value, p value, expected counts')
cs1= scipy.stats.chi2_contingency(ct1)
print (cs1)

### Large X2 statistics and a small p-value states that there is a strong association between the explanatory variables and the response variables as opposed to the null hypothesis.

In [ ]:
seaborn.factorplot(x= 'GDPCat', y= 'Mortal', data= data4, kind= "bar", ci=None)
plt.xlabel('GDP per capita category: 1 the most poor to 6 the richest')
plt.ylabel('Proportion mortality')
plt.show()
# note that the y values are equal to the column percentages (colpct) for Mortal 1

### POST-HOC TESTS ARE NEEDED:  Which GDP Categories are different than the others?
### chi square post hoc test: Apply chi square for each pair (all combinations of them); test with adjusted p value (bonferroni adjustment p-value: p-value / number_of_comparions)
Here, we have 6 groups and there are 15 pairs; so the adjusted p-value is 0.05/15= 0,0033

In [ ]:
# We need to run chi-square tests for each of the comparisons
recode_1v2 = {1: 1, 2: 2} # this is required because we are going to use only these two columns
data4['GDP_1v2']= data4['GDPCat'].map(recode_1v2)

# contingency table of observed counts
table_1v2=pd.crosstab(data4['Mortal'], data4['GDP_1v2'])
print (table_1v2)

# column percentages
colsum=table_1v2.sum(axis=0)
colpct=table_1v2/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_1v2= scipy.stats.chi2_contingency(table_1v2)
print (cs_1v2)

#### Now, we need to run the above test for each of the remaining paired comparisons!

In [ ]:
recode_1v3 = {1: 1, 3: 3}
data4['GDP_1v3']= data4['GDPCat'].map(recode_1v3)

# contingency table of observed counts
table_1v3=pd.crosstab(data4['Mortal'], data4['GDP_1v3'])
print (table_1v3)

# column percentages
colsum=table_1v3.sum(axis=0)
colpct=table_1v3/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_1v3= scipy.stats.chi2_contingency(table_1v3)
print (cs_1v3)

In [ ]:
recode_1v4 = {1: 1, 4: 4}
data4['GDP_1v4']= data4['GDPCat'].map(recode_1v4)

# contingency table of observed counts
table_1v4=pd.crosstab(data4['Mortal'], data4['GDP_1v4'])
print (table_1v4)

# column percentages
colsum=table_1v4.sum(axis=0)
colpct=table_1v4/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_1v4= scipy.stats.chi2_contingency(table_1v4)
print (cs_1v4)

In [ ]:
recode_1v5 = {1: 1, 5: 5}
data4['GDP_1v5']= data4['GDPCat'].map(recode_1v5)

# contingency table of observed counts
table_1v5=pd.crosstab(data4['Mortal'], data4['GDP_1v5'])
print (table_1v5)

# column percentages
colsum=table_1v5.sum(axis=0)
colpct=table_1v5/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_1v5= scipy.stats.chi2_contingency(table_1v5)
print (cs_1v5)

In [ ]:
recode_1v6 = {1: 1, 6: 6}
data4['GDP_1v6']= data4['GDPCat'].map(recode_1v6)

# contingency table of observed counts
table_1v6=pd.crosstab(data4['Mortal'], data4['GDP_1v6'])
print (table_1v6)

# column percentages
colsum=table_1v6.sum(axis=0)
colpct=table_1v6/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_1v6= scipy.stats.chi2_contingency(table_1v6)
print (cs_1v6)

In [ ]:
recode_2v3 = {2: 2, 3: 3}
data4['GDP_2v3']= data4['GDPCat'].map(recode_2v3)

# contingency table of observed counts
table_2v3=pd.crosstab(data4['Mortal'], data4['GDP_2v3'])
print (table_2v3)

# column percentages
colsum=table_2v3.sum(axis=0)
colpct=table_2v3/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_2v3= scipy.stats.chi2_contingency(table_2v3)
print (cs_2v3)

In [ ]:
recode_2v4 = {2: 2, 4: 4}
data4['GDP_2v4']= data4['GDPCat'].map(recode_2v4)

# contingency table of observed counts
table_2v4=pd.crosstab(data4['Mortal'], data4['GDP_2v4'])
print (table_2v4)

# column percentages
colsum=table_2v4.sum(axis=0)
colpct=table_2v4/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_2v4= scipy.stats.chi2_contingency(table_2v4)
print (cs_2v4)

In [ ]:
recode_2v5 = {2: 2, 5: 5}
data4['GDP_2v5']= data4['GDPCat'].map(recode_2v5)

# contingency table of observed counts
table_2v5=pd.crosstab(data4['Mortal'], data4['GDP_2v5'])
print (table_2v5)

# column percentages
colsum=table_2v5.sum(axis=0)
colpct=table_2v5/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_2v5= scipy.stats.chi2_contingency(table_2v5)
print (cs_2v5)

In [ ]:
recode_2v6 = {2: 2, 6: 6}
data4['GDP_2v6']= data4['GDPCat'].map(recode_2v6)

# contingency table of observed counts
table_2v6=pd.crosstab(data4['Mortal'], data4['GDP_2v6'])
print (table_2v6)

# column percentages
colsum=table_2v6.sum(axis=0)
colpct=table_2v6/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_2v6= scipy.stats.chi2_contingency(table_2v6)
print (cs_2v6)

In [ ]:
recode_3v4 = {3: 3, 4: 4}
data4['GDP_3v4']= data4['GDPCat'].map(recode_3v4)

# contingency table of observed counts
table_3v4=pd.crosstab(data4['Mortal'], data4['GDP_3v4'])
print (table_3v4)

# column percentages
colsum=table_3v4.sum(axis=0)
colpct=table_3v4/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_3v4= scipy.stats.chi2_contingency(table_3v4)
print (cs_3v4)

In [ ]:
recode_3v5 = {3: 3, 5: 5}
data4['GDP_3v5']= data4['GDPCat'].map(recode_3v5)

# contingency table of observed counts
table_3v5=pd.crosstab(data4['Mortal'], data4['GDP_3v5'])
print (table_3v5)

# column percentages
colsum=table_3v5.sum(axis=0)
colpct=table_3v5/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_3v5= scipy.stats.chi2_contingency(table_3v5)
print (cs_3v5)

In [ ]:
recode_3v6 = {3: 3, 6: 6}
data4['GDP_3v6']= data4['GDPCat'].map(recode_3v6)

# contingency table of observed counts
table_3v6=pd.crosstab(data4['Mortal'], data4['GDP_3v6'])
print (table_3v6)

# column percentages
colsum=table_3v6.sum(axis=0)
colpct=table_3v6/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_3v6= scipy.stats.chi2_contingency(table_3v6)
print (cs_3v6)

In [ ]:
recode_4v5 = {4: 4, 5: 5}
data4['GDP_4v5']= data4['GDPCat'].map(recode_4v5)

# contingency table of observed counts
table_4v5=pd.crosstab(data4['Mortal'], data4['GDP_4v5'])
print (table_4v5)

# column percentages
colsum=table_4v5.sum(axis=0)
colpct=table_4v5/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_4v5= scipy.stats.chi2_contingency(table_4v5)
print (cs_4v5)

In [ ]:
recode_4v6 = {4: 4, 6: 6}
data4['GDP_4v6']= data4['GDPCat'].map(recode_4v6)

# contingency table of observed counts
table_4v6=pd.crosstab(data4['Mortal'], data4['GDP_4v6'])
print (table_4v6)

# column percentages
colsum=table_4v6.sum(axis=0)
colpct=table_4v6/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_4v6= scipy.stats.chi2_contingency(table_4v6)
print (cs_4v6)

In [ ]:
recode_5v6 = {5: 5, 6: 6}
data4['GDP_5v6']= data4['GDPCat'].map(recode_5v6)

# contingency table of observed counts
table_5v6=pd.crosstab(data4['Mortal'], data4['GDP_5v6'])
print (table_5v6)

# column percentages
colsum=table_5v6.sum(axis=0)
colpct=table_5v6/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs_5v6= scipy.stats.chi2_contingency(table_5v6)
print (cs_5v6)

### A table showing the p-values obtained for each group should be drawn as shown below:

In [ ]:
from IPython.display import Image
Image("table1.png")

## C) PEARSON CORRELATION BETWEEN TWO NUMERIC VARIABLES:

We shall see this later in another project...